In [1]:
import tensorly as tl
import numpy as np
from numpy import linalg as la
from sympy import *
from sympy import Matrix, symbols, solve_linear_system
from sympy.solvers.solveset import linsolve
import time
from joblib import Parallel, delayed
x,y,z_1,z_2,z_3 = symbols('x y z_1 z_2 z_3')
x,y,z1,z2,z3 ,b= symbols("x,y,z1,z2,z3,b")
p_112, p_102, p_002, p_012 = symbols('p_112 p_102 p_002 p_012')
p_000, p_001, p_100, p_101, p_200, p_201, p_010, p_011, p_020, p_021, p_110, p_111, p_210, p_211, p_120, p_121, p_220, p_221 = symbols('p_000 p_001 p_100 p_101 p_200 p_201 p_010 p_011 p_020 p_021 p_110 p_111 p_210 p_211 p_120 p_121 p_220 p_221')

In [2]:
#generates random rank 3 tensors
def rank_tree():
    return (low_tensor() + low_tensor() + low_tensor()) 

# random tensor of some dim
def rand_tensor():
    return tl.tensor(np.random.randint(1, 100000, size=(3,2,2)))*1.0

# generates rank 1 tensors
def low_tensor():
    max = 30
    a = np.random.randint(1, max, size=3) 
    b = np.random.randint(1, max, size=2)
    c = np.random.randint(1, max, size=2)
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(3, 2, 2)) * 1.0
    return tens

# checks if all subtensors have nonneg rank 3
def check(t):
    t1 = tl.tensor([t[0], t[1]])
    t2 = tl.tensor([t[1], t[2]])
    t3 = tl.tensor([t[0], t[2]])
    a1 = det(Matrix(t1[0]))
    a2 = det(Matrix(t1[1]))
    a3 = det(Matrix(t2[0]))
    a4 = det(Matrix(t2[1]))
    b1 = det(Matrix(t1[:,0]))
    b2 = det(Matrix(t1[:,1]))
    b3 = det(Matrix(t2[:,0]))
    b4 = det(Matrix(t2[:,1]))
    c1 = det(Matrix(t1[:,:,0]))
    c2 = det(Matrix(t1[:,:,1]))
    c3 = det(Matrix(t2[:,:,0]))
    c4 = det(Matrix(t2[:,:,1]))

    a5 = det(Matrix(t3[0]))
    a6 = det(Matrix(t3[1]))

    b5 = det(Matrix(t3[:,0]))
    b6 = det(Matrix(t3[:,1]))

    c5 = det(Matrix(t3[:,:,0]))
    c6 = det(Matrix(t3[:,:,1]))
    return sgn([a1, a2,a3,a4,a5,a6]) or sgn([b1,b2,b3,b4,b5,b6]) or sgn([c1,c2,c3,c4,c5,c6])

# determines sign of a list
def sgn(a):
    t = 0
    ab = 0
    for a_i in a:
        t+= abs(a_i)
        ab += a_i
    return t == abs(ab)


# checks if the give 2x2x2 tensor has nonneg rank 3
def check_r3(t):
    a1 = det(Matrix(t[0]))
    a2 = det(Matrix(t[1]))
    b1 = det(Matrix(t[:,0]))
    b2 = det(Matrix(t[:,1]))
    c1 = det(Matrix(t[:,:,0]))
    c2 = det(Matrix(t[:,:,1]))
    return sgn([a1,a2]) or sgn([b1,b2]) or sgn([c1,c2])

# checks if given 2x2x2 has nonneg rank 2
def check_r2(t):
    a1 = det(Matrix(t[0]))
    a2 = det(Matrix(t[1]))
    b1 = det(Matrix(t[:,0]))
    b2 = det(Matrix(t[:,1]))
    c1 = det(Matrix(t[:,:,0]))
    c2 = det(Matrix(t[:,:,1]))
    d1 = ineq(a1,a2,b1,b2,c1,c2,ge,ge,ge)
    d2 = ineq(a1,a2,b1,b2,c1,c2,le,le,ge)
    d3 = ineq(a1,a2,b1,b2,c1,c2,le,ge,le)
    d4 = ineq(a1,a2,b1,b2,c1,c2,ge,le,le)
    supermod = d1 or d2 or d3 or d4
    return supermod


# helpers
def ineq(a1,a2,b1,b2,c1,c2,f1,f2,f3):
    t1 = f1(a1,0) and f1(a2,0)
    t2 = f2(b1,0) and f2(b2,0)
    t3 = f3(c1,0) and f3(c2,0)
    return t1 and t2 and t3
    
def ge(a1,a2):
    return a1 >= a2

def le(a1,a2):
    return a1 <= a2

# Checks if 2x2x3 contains a 2x2x2 with nonneg 4
def check_r4(t):
    a1 = not check_r3(tl.tensor([t[0],t[1]]))
    a2 = not check_r3(tl.tensor([t[0],t[2]]))
    a3 = not check_r3(tl.tensor([t[1],t[2]]))
    return a1 or a2 or a3 

In [7]:
tens = rank_tree()
#tens = tl.tensor(np.random.randint(1, 100000, size=(3,2,2)))*1.0
M_b = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,1])]]).transpose()
M_a = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,0])]]).transpose()
a = 1
b = M_b.det() / M_a.det()
M = Matrix([[tens[0][0][0],0,a,0,0],[0,tens[0][0][1],b,0,0],
            [tens[1][0][0],0,0,a,0],[0,tens[1][0][1],0,b,0],
            [tens[2][0][0],0,0,0,a],[0,tens[2][0][1],0,0,b]])
R = Matrix([Matrix(tens[0,1]),Matrix(tens[1,1]),Matrix(tens[2,1])])
M_sub = Matrix(M[0:5,0:5])
sol = np.array((M_sub.adjugate() @ Matrix(R[0:5])) /M_sub.det(), dtype = "float")
sol1 = la.solve(np.array(M.T @ M, dtype = "float"), np.array(M.T @ R, dtype = "float"))
sol


array([[ 2.24165872e+00],
       [ 2.37893353e+00],
       [-2.57185224e+03],
       [-1.75666397e+04],
       [-7.10856864e+03]])

In [10]:
proc_tensor(tens)

array([ 3.24424628e-01,  2.24165872e+00,  2.37893353e+00, -2.57185224e+03,
       -1.75666397e+04, -7.10856864e+03])

In [21]:
def mat_inv(t):
    tens = t.copy()
    M_1 = Matrix([Matrix(tens[0][1]).transpose(),Matrix(tens[0][0]).transpose()])
    M_2 = Matrix([Matrix(tens[1][1]).transpose(),Matrix(tens[1][0]).transpose()])
    M_3 = Matrix([Matrix(tens[2][1]).transpose(),Matrix(tens[2][0]).transpose()])
    tens[0] = M_1
    tens[1] = M_2
    tens[2] = M_3
    return tens

def mat_trans(t):
    tens = t.copy()
    tens[0] = tens[0].transpose()
    tens[1] = tens[1].transpose()
    tens[2] = tens[2].transpose()
    return tens

def proc_tensor(t):
    tens = t.copy()
    M_b = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,1])]]).transpose()
    M_a = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,0])]]).transpose()
    a = 1
    b = M_b.det() / M_a.det()
    M = Matrix([[tens[0][0][0],0,a,0,0],[0,tens[0][0][1],b,0,0],
                [tens[1][0][0],0,0,a,0],[0,tens[1][0][1],0,b,0],
                [tens[2][0][0],0,0,0,a],[0,tens[2][0][1],0,0,b]])
    R = Matrix([Matrix(tens[0,1]),Matrix(tens[1,1]),Matrix(tens[2,1])])
    M_sub = Matrix(M[0:5,0:5])
    sol = np.array(M_sub.adjugate() @ Matrix(R[0:5]) /M_sub.det(), dtype = "float")
    return np.append(np.array([b], dtype = "float"),sol)

def loop_rotations(i):
    tens = tl.tensor(np.random.randint(1, 100000, size=(3,2,2)))*1.0
    #tens = rank_tree()
    ret = [0,0,0,0]
    c1 = proc_tensor(tens)
    if np.all(c1>=0):
        return (1,0)
    c2 = proc_tensor(mat_inv(tens))
    if np.all(c2>=0):
        return (1,0)
    c3 = proc_tensor(mat_trans(tens))
    if np.all(c3>=0):
        return (1,0)
    c4 = proc_tensor(mat_inv(mat_trans(tens)))
    if np.all(c4>=0):
        return (1,0)
    if check_r4(tens):
        return (0,1)
    return (0,0)

In [22]:
te = time.time()
total = 1000
results = Parallel(n_jobs=4)(delayed(loop_rotations)(i) for i in range(total))
res = [sum(x) for x in zip(*results)]
print(res)
print(time.time() - te)

[146, 500]
36.48608088493347


In [172]:
proc_tensor(mat_inv(tens))

array([  5.66666667,   0.78571429,   0.78571429, 281.14285714,
       632.57142857, 210.85714286])

In [173]:
proc_tensor(mat_trans(tens))

array([ 1.27272727e+00,  5.66666667e+00,  5.66666667e+00, -2.10100000e+03,
       -4.44766667e+03, -2.23190000e+04])

In [174]:
proc_tensor(mat_inv(mat_trans(tens)))

array([1.27272727e+00, 1.76470588e-01, 1.76470588e-01, 3.70764706e+02,
       7.84882353e+02, 3.93864706e+03])

In [175]:
tens

array([[[  927.,  3152.],
        [  822.,  1984.]],

       [[ 2017.,  6982.],
        [ 1762.,  4324.]],

       [[ 5604.,  9437.],
        [ 6864., 10490.]]])

array([[[  927.,   822.],
        [ 3152.,  1984.]],

       [[ 2017.,  1762.],
        [ 6982.,  4324.]],

       [[ 5604.,  6864.],
        [ 9437., 10490.]]])

In [197]:
def proc_new(i):
    tens = rank_tree()
    tens = tl.tensor(np.random.randint(1, 100000, size=(3,2,2)))*1.0
    M_b = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,1])]]).transpose()
    M_a = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,0])]]).transpose()
    a = 1
    b = M_b.det() / M_a.det()
    M = Matrix([[tens[0][0][0],0,a,0,0],[0,tens[0][0][1],b,0,0],
                [tens[1][0][0],0,0,a,0],[0,tens[1][0][1],0,b,0],
                [tens[2][0][0],0,0,0,a],[0,tens[2][0][1],0,0,b]])
    R = Matrix([Matrix(tens[0,1]),Matrix(tens[1,1]),Matrix(tens[2,1])])
    M_sub = Matrix(M[0:5,0:5])
    sol = np.array(M_sub.adjugate() * Matrix(R[0:5]) /M_sub.det(), dtype = "float")
    #display(Latex(latex(M1)))
    ret = [0,0,0,0]
    cond = np.all(sol >= 0) and a >= 0 and b >= 0
    cond1 = check_r4(tens)
    if np.all(sol > 0) and b >= 0:
        ret[3] = 1
    if cond and cond1:
        ret[0] = 1
    if cond:
        ret[1] = 1
    if cond1:
        ret[2] = 1
    return tuple(ret)

In [203]:
te = time.time()
total = 100
results = Parallel(n_jobs=4)(delayed(proc_new)(i) for i in range(total))
res = [sum(x) for x in zip(*results)]
print(res)
print(time.time() - te)

[0, 5, 53, 5]
1.0617210865020752


In [69]:
499/10000

0.0499

In [145]:
tens = rand_tensor()
tens

array([[[82580., 17454.],
        [10171., 65847.]],

       [[53559.,  5176.],
        [91069.,  2536.]],

       [[93689., 42512.],
        [47881., 31643.]]])

In [149]:
mat_inv(mat_trans(tens))

array([[[17454., 65847.],
        [82580., 10171.]],

       [[ 5176.,  2536.],
        [53559., 91069.]],

       [[42512., 31643.],
        [93689., 47881.]]])

In [130]:
tens[0] = tens[0].transpose()
tens[1] = tens[1].transpose()
tens[2] = tens[2].transpose()
tens

array([[[41198., 82052.],
        [73393., 90038.]],

       [[12086., 56125.],
        [ 2736., 74152.]],

       [[52246., 46756.],
        [78573., 61659.]]])

In [133]:
T = Matrix(tens[0,1].transpose())
T

Matrix([
[73393.0],
[90038.0]])

In [137]:
M = Matrix([Matrix(tens[1][1]).transpose(),Matrix(tens[1][0]).transpose()])
M

Matrix([
[ 2736.0, 74152.0],
[12086.0, 56125.0]])

In [144]:
mat_trans(tens)

array([[[73393., 41198.],
        [90038., 82052.]],

       [[ 2736., 12086.],
        [74152., 56125.]],

       [[78573., 52246.],
        [61659., 46756.]]])